# Neural Networks with Tensorflow

In [ ]:
import addutils.toc ; addutils.toc.js(ipy_notebook=True)

In [ ]:
import numpy as np
import tensorflow as tf
from time import time
from IPython.display import Image
import math
from tensorflow.examples.tutorials.mnist import input_data
from addutils import css_notebook
css_notebook()

In [ ]:
import bokeh.plotting as bk
from bokeh.io import push_notebook
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
bk.output_notebook()

## 1 Neural Networks

**TODO**: introduzione all NN, perché le usiamo? Migliorare il risultato rispetto alla logistic regression di prima.

### 1.1 Neural Network Basics

What is a neuron?

A brain neuron has a structure formed by a cell body, an axon were it sends messages to other neuron and dendritic tree were it receives messages from other neuron. A neuron generates outgoing charge (throughout the axon) whenever enough charge has been received (activation).

In [ ]:
Image('images/neuron.jpeg', width=400, height=400)

In neural networks we use an approximation of neurons that look like this:

In [ ]:
Image('images/Simple Neural Network.png',  width=500, height=500)

In practice they learn how to approximate functions. For example, for a classifier, it maps an input to a category $y = f(X)$. They are called networks because they combine many function together. The graph depicts how the function are composed in layers. Each function in the most basic form combines the input with a series of weights. Think about the most basic linear regression $y = W^TX+b$. $w$ is called the **weights matrix** (if we are in higher dimensions) and $b$ is the **bias vector**. If we think about the neurons of the brain are activated by a stimuli, it is not suprising that an **activation function** can be applyed to the linear combination of weights, biases and inputs. 

One example of activation function is the sigmoid that we have seen in the previous notebook.

In real life, neurons are much more complex than this, and there are many types of neural networks. So the analogy with the brain stops here. 

### 1.2 Architecture of Neural Networks

**TODO** definire l'architettura

In [ ]:
Image('images/neural_net.jpeg')

To define the number of layers in a network we count the number of layers that have weigths. This is a two layer network, that has one hidden layer.

The reason we use layers is that allows us to implement computation more efficiently, with matrix multiplication.

The result of all linear combinations and activation functions gives the output, and to quantify how the Neural Network is performing we define a **cost function** $C$. We have already seen two loss functions, MSE and Cross Entropy. In next section we will use MSE to understand how backpropagation work.

### 1.4 Backpropagation

#### 1.4.1 Gradient Descent

Learning in this contex reduces to finding the best weights and biases that allow the neural network to approximate the function, that is to have a small cost for all training inputs $X$. 

For the moment we will use the MSE as the cost function, but the same reasoning can be applyed to all type of cost functions. 
$$
C(W,b) = \frac{1}{2n}\sum_x \|y-f(X)\|^2
$$
Here $W$ and $b$ denotes all the weights and biases in the network, $n$ is the number of training samples in the data and $f(X)$ is the result of applying the forward pass of the network with input $X$. 

We see that $C(W,b)$ is non-negative, since every term in the sum is non-negative, furthermore, the cost becomes small, when $f(X)$ is approximately equal to the target, for all training inputs. By contrast it is not performing well when $C(W,b)$ is large, that is $f(X)$ is not close to the target for a large number of inputs. The purpose of  the training algorithm is to minimize the cost $C(W,b)$ as a function of the weights and biases. The algorithm that precisely does this is called **gradient descent**.

Quadratic cost works well in this settings, because it is a smooth function, that is making small changes to the weights and biases changes the output accordingly. That makes easy to figure out the amount of change to apply to weights and biases. 

This is a well posed problem but at the moment we are ignoring all the structure of the network, the activation function, weights and biases and so on. It turns out that we can understand a tremendous amount by ignoring most of that structure, and just concentrating on the minimization aspect. So for now we're going to forget all about the specific form of the cost function, the connection to neural networks, and so on. Instead, we're going to minimize some function, $C(v)$. This could be any real-valued function of many variables, $v=v1,v2,\ldots$. It could help to imagine $C$ as a function of just two variables: $v1$ and $v2$:

In [ ]:
Image('images/valley.png',  width=500, height=500)

The objective of our algorithm is to find where $C$ achieves its global minimum. In this simple context it is easy, but in Neural Networks $C$ can be a complicated function of many variables. 

One way to solve the problem is to use calculus to try to find the minimum analytically, by computing derivatives and using them to find places where C is an extremum. This approach can be feasible when the function is of just two variables but it can be really hard when the function is composed of many variables (like that of Neural Networks), so using calculus in this case simply won't work.

Luckily gradient descent doesn't require calculus and works pretty well in this context. If we think of our function as a kind of a valley and we imagine a ball rolling down the slope of the valley, the ball will eventually roll to the bottom of the valley. How can we use this analogy to devise a method for finding the minimum of $C$? Let's think about what happens when we move the ball a small amount $\Delta v_1$ in the $v_1$ direction, and a small amount $\Delta v_2$ in the $v_2$ direction. $C$ changes as follows: 

$$\Delta C \approx \frac{\partial C}{\partial v_1} \Delta v_1 + \frac{\partial C}{\partial v_2} \Delta v_2.$$

We're going to find a way of choosing $\Delta v_1$ and $\Delta v_2$ so as to make $\Delta C$ negative, so the ball is rolling down into the valley. 

To figure out how to make such a choice it helps to define $\Delta v$ to be the vector of changes in $\Delta v \equiv (\Delta v_1, \Delta v_2)^T$. We'll also define the gradient of $C$ to be the vector of partial derivatives

$$
  \nabla C \equiv \left( \frac{\partial C}{\partial v_1}, 
  \frac{\partial C}{\partial v_2} \right)^T
$$

With these definitions, the expression for $\Delta C$ can be rewritten as $\Delta C \approx \nabla C \cdot \Delta v$, $\Delta C$ relates changes in $v$ to changes in $C$ and it lets us see how to choose $\Delta v$ so as to make $\Delta C$ negative. In particular, suppose we choose 

$$
  \Delta v = -\eta \nabla C
$$

then $\Delta C \approx -\eta \nabla C \cdot \nabla C = -\eta \|\nabla C\|^2$. Because $\|\nabla C\|^2 > 0$, this guarantees that C will always decrease, never increase, if we change $v$ according to equation above. That is exactly what we wanted (but of course remeber that is an approximation). We'll then use this equation to compute a value for $\Delta v$, then move the ball's position $v$ by that amount:

$$
  v \rightarrow v' = v -\eta \nabla C.
$$

Then we'll use this update rule again, to make another move. If we keep doing this, over and over, we'll keep decreasing C until - we hope - we reach a global minimum.

Summing up, the way the gradient descent algorithm works is to repeatedly compute the gradient $\nabla C$, and then to move in the opposite direction, "falling down" the slope of the valley. We can visualize it like this:

In [ ]:
Image('images/valley_with_ball.png',  width=500, height=500)

To make gradient descent work correctly, we need to choose the learning rate $\eta$ to be small enough that equation $\Delta C \approx \nabla C \cdot \Delta v$ is a good approximation. In practice a lower value of $\eta$ will make small changes in $\Delta v$, while an higher value could even end us up in a worse position $\Delta C > 0$. Setting the right value of $\eta$ is tricky and in certain situations can vary over time. 

You can think of this update rule as defining the gradient descent algorithm. It gives us a way of repeatedly changing the position $v$ in order to find a minimum of the function $C$. 

How can we apply gradient descent to learn in a neural network? Position $v$ now has components $w_k$ and $b_l$, and the gradient vector $\nabla C$ has corresponding components $\partial C / \partial w_k$ and $\partial C/ \partial b_l$. Writing out the gradient descent update rule in terms of components, we have:

$$
\begin{eqnarray}
  w_k & \rightarrow & w_k' = w_k-\eta \frac{\partial C}{\partial w_k} \\
  b_l & \rightarrow & b_l' = b_l-\eta \frac{\partial C}{\partial b_l}
\end{eqnarray}
$$

By repeatedly applying this update rule we can "roll down the hill", and hopefully find a minimum of the cost function. In other words, this is a rule which can be used to learn in a neural network.

Notice that the MSE cost function has the form $C = \frac{1}{n} \sum_x C_x$, that is, it's an average over costs $C_x \equiv \frac{\|y - f(x)\|^2}{2}$ for individual training examples. In practice, to compute the gradient $\nabla C$ we need to compute the gradients $\nabla C_x$ separately for each training input, x, and then average them, $\nabla C = \frac{1}{n}
\sum_x \nabla C_x$.

#### 1.4.2 Backpropagation

In previous section we talked about gradient descent,but so far we applyed it (in previous notebook to a simple case where the parameters to optimize where only one weight vector and a bias term. What happens in a more complicated setting, for example that of a neural network? How does gradient descent work in this setting? 

To train a Neural Network in practice we use an algorithm called **backpropagation** that calculate the exact contribute of each weight of the network (that is its gradient) to the final output and update each weight (and bias) accordingly. 

In [ ]:
Image('images/gradients.png',  width=500, height=500)

This is a neuron in a computational graph of a Neural Network. Suppose we are calculating the output given a particular set of inputs $x$ and $y$. The output $z$ is passed **forward** until it reaches the output and the loss is calculated. Now we need a way to propagate the error **backward** until we can find the gradient for $x$ and $y$. Suppose that we have already the partial derivative of $z$ w.r.t. the loss $L$. What is the partial derivative of $x$ w.r.t. to $L$? Well it depdends on $f$ for sure, but since it is a composite function we can decompose its derivative with the **chain rule** that is 

$$ \frac{\partial L}{\partial x} = \frac{\partial L}{\partial z} \frac{\partial z}{\partial x}$$

The important thing to notice is that the local derivatives $\frac{\partial z}{\partial x}$ and $\frac{\partial z}{\partial y}$ can be computed right away when we recive the inputs $x$ and $y$. Then the whole process can be repeated recursively.

Then when we have all the gradients calcluated for the whole graph we can update the weights. Tensorflow does that automatically, because it knows how to compute the derivative of $f$ for each function that can be used as activation!

Remeber that $x$ and $y$ in neural networks are tensors and weights are matrices.

#### 1.4.3 Optimization functions

When the number of training inputs is very large gradient descent can take a long time, and learning thus occurs slowly.

An idea called **stochastic gradient descent** can be used to speed up learning. The idea is to estimate the gradient by computing them for a small sample of randomly chosen training inputs. By averaging over this small sample it turns out that we can quickly get a good estimate of the true gradient, and this helps speed up gradient descent, and thus learning.

To make these ideas more precise, stochastic gradient descent works by randomly picking out a small number m of randomly chosen training inputs. We refer to them as a **mini-batch**. Provided the sample size is large enough we expect that the average value of the gradient will be roughly equal to the average overall.

### 1.4 Activation functions

There are many activation functions, perhaps two of the most popular ad the sigmoid function: 
$$
\sigma(x) = \frac{1}{1+e^{-z}}
$$

and the hyperbolic tangent: 

$$
tanh(x)
$$

Recently (2012) another activation function has become quite popular, Rectifier Linear Unit (ReLU): 

$$
max(0, x)
$$

and it is the current default choice in many modern network architectures.

In [ ]:
Image('images/activation_functions.png',  width=800, height=800)

### 1.5 Additional Resources

* [TensorFlow playground](http://playground.tensorflow.org/)
* [TensorFlow examples](https://github.com/aymericdamien/TensorFlow-Examples)
* [Colah's Blog](http://colah.github.io/)
* [Neural Networks and Deep Learning](http://neuralnetworksanddeeplearning.com)
* [distill.pub](https://distill.pub/)

## 2 Classification with a Multi Layer Perceptron

### 2.1 The Data

In [ ]:
mnist = input_data.read_data_sets('example_data/MNIST_data', one_hot=True)

In [ ]:
# The MNIST dataset has 10 classes, representing the digits 0 through 9.
NUM_CLASSES = 10

# The MNIST images are always 28x28 pixels.
IMAGE_SIZE = 28
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE

#### 2.1.1 Helper functions

In [ ]:
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

### 2.2 The Network

Now we will build the network step-by-step, we will discuss its (simple) architecture and how to built it with TensorFlow.

Our simple MLP looks like this: input -> hidden layer -> output classification

Defining a `tf.placeholder` with `None` as its first dimension indicates that the first dimension, corresponding to the batch size, can be any size. These placeholder will contain input images and labels and will be fed to the program during the training phase.

In [ ]:
image_placeholder = tf.placeholder(tf.float32, shape=[None, IMAGE_PIXELS])
label_placeholder = tf.placeholder(tf.int32, shape=[None, NUM_CLASSES])

In order to provide a dictionary when feeding the placeholder, we use a small helper function.

In [ ]:
def get_feed_dict(batch, place_1, place_2):
    return {place_1: batch[0], place_2: batch[1]}

The hidden layer has a number of units that can be specified with the parameter `hidden1_units` and it computes a transformation of the linear combination of inputs, weights and biases. There are a lot of transformation available in TensorFlow, but for this example we will use ReLU.

The weight matrix is initialized to random values, while the bias vector is initialized to a costant values (a smallquantity, it can be zero); it is also good practice to initialize weights of a ReLU with a slightly positive initial bias to avoid "dead neurons". A reasonable-sounding idea then might be to set all the initial weights to zero, which we expect to be the "best guess" in expectation. This turns out to be a mistake, because if every neuron in the network computes the same output, then they will also all compute the same gradients during backpropagation and undergo the exact same parameter updates. In other words, there is no source of asymmetry between neurons if their weights are initialized to be the same. It is possible and common to initialize the biases to be zero, since the asymmetry breaking is provided by the small random numbers in the weights

In [ ]:
hidden1_units = 32

with tf.name_scope('hidden1'):
    weights = tf.Variable(
        tf.truncated_normal([IMAGE_PIXELS, hidden1_units], 
                            stddev=0.1 / math.sqrt(float(IMAGE_PIXELS))),
        name='weights')
    variable_summaries(weights)
    biases = tf.Variable(tf.zeros([hidden1_units]),
                         name='biases')

    variable_summaries(weights)
    hidden1 = tf.nn.relu(tf.matmul(image_placeholder, weights) + biases)

Note the use of `tf.name_scope` to define a prefix to the variables. 

The last stage compute a **softmax** transformation of the hidden layer. 

In [ ]:
with tf.name_scope('softmax_linear'):
    weights = tf.Variable(
        tf.truncated_normal([hidden1_units, NUM_CLASSES],
                            stddev=1.0 / math.sqrt(float(hidden1_units))),
        name='weights')
    variable_summaries(weights)
    biases = tf.Variable(tf.zeros([NUM_CLASSES]),
                         name='biases')
    variable_summaries(biases)
    logits = tf.matmul(hidden1, weights) + biases
    tf.summary.histogram('logits', logits)

Now it is possible to use TensorFlow automatic differentiation to find the gradients of the cost with respect to each variable. For this example we use basic steepest gradient descent as optimizer.  **Gradient Descent** is a simple procedure, where TensorFlow simply shifts each variable a little bit in the direction that reduces the cost. This *little bit* is the **learning rate** and it is multiplied by the negative of the gradient. For this examples the learning rate is fixed at 0.05. Actually we are using **Stochastic Gradient Descent**. Stochastic means that we are not using all the data to perform a single update of the weights but we are using a subsample at a time. This subsample is called **mini batch** and it is random. Each time we exahaust the training set we can recompute another subsample and so on. Each iteration (that is each time the algorithm consumes all the dataset) is an epoch. At each epoch the function gets better.

Next we define the cost function, cross entropy in this case. 

In [ ]:
learning_rate = 0.01

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                        labels=label_placeholder,
                                                        name='xentropy')
loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
tf.summary.scalar('xentropy', loss)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(loss)

To evaluate the model we first compute the position of the maximum entry in both predicted and real output. This corresponds to the class to which the example belongs. Then we compare the two vector to determine what fraction of the predicted output is correct.

In [ ]:
correct_prediction = tf.equal(tf.argmax(tf.nn.softmax(logits),1), 
                              tf.argmax(label_placeholder,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge_all()

After we constructed the graph, we need to start a session to run it. 

In [ ]:
sess = tf.Session()

In [ ]:
train_writer = tf.summary.FileWriter('temp/mnist_fc' + '/train',
                                      sess.graph)
test_writer = tf.summary.FileWriter('temp/mnist_fc' + '/test')

Next thing to do is to initialize the variable in the session.

In [ ]:
sess.run(tf.global_variables_initializer())

Ok, now we are ready to launch the training:

In [ ]:
MAX_STEPS = 5000
BATCH_SIZE = 128

for i in range(MAX_STEPS):
    feed_dict = get_feed_dict(mnist.train.next_batch(BATCH_SIZE), 
                              image_placeholder, 
                              label_placeholder)
    summary, _ = sess.run([merged, train_op], feed_dict=feed_dict)
    train_writer.add_summary(summary, i)
    if i % 500 == 0:  
        feed_dict = get_feed_dict(mnist.validation.next_batch(BATCH_SIZE), 
                                  image_placeholder, 
                                  label_placeholder)
        summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict)
        test_writer.add_summary(summary, i)
        print('Accuracy at step %s: %s' % (i, acc))

**REMARK** Epochs are calculated directly by the mnist object. Whenever the number of batches requested is greather than the number of examples it automatically updates the number of epochs. The object shuffles also the training set for better learning.

In [ ]:
print(accuracy.eval(session=sess, feed_dict={image_placeholder: mnist.test.images, 
                                             label_placeholder: mnist.test.labels}))

In [ ]:
sess.close()

### 2.3 Tensorboard

>The computations you'll use TensorFlow for - like training a massive deep neural network - can be complex and confusing. To make it easier to understand, debug, and optimize TensorFlow programs, we've included a suite of visualization tools called TensorBoard. You can use TensorBoard to visualize your TensorFlow graph, plot quantitative metrics about the execution of your graph, and show additional data like images that pass through it. 

In a terminal type: `tensorboard --logdir='temp/mnist_fc` and follow instructions.

## 3 Simplify TensorFlow

### 3.1 TensorFlow contrib.learn

The module **learn** in the contrib section of the repository of TensorFlow was formerly known as skflow and it is meant to provide ease of use and modularity, for beginners and for integrating TensorFlow models in existing code. It has an interface similar to scikit-learn and it is possible to create a Neural Network with few line of code.

In [ ]:
from tensorflow.contrib import learn
tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
classifier = learn.DNNClassifier(hidden_units=[hidden1_units, 20, 10],
                                 n_classes=NUM_CLASSES,
                                 feature_columns=learn.infer_real_valued_columns_from_input(mnist.train.images.astype(np.float32)),
                                 optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.05)) 

With just one line of code we are able to create the same neural network that took us many line of code to compute, and with just one line we can fit it.

In [ ]:
classifier.fit(mnist.train.images, 
               np.argmax(mnist.train.labels, axis=1).astype(np.int64), 
               batch_size=BATCH_SIZE, 
               steps=3000) 

The accuracy is similar to the example above.

In [ ]:
accuracy_score = classifier.evaluate(x=mnist.test.images.astype(np.float32), 
                                     y=np.argmax(mnist.test.labels, axis=1).astype(np.int64))["accuracy"]
print('Accuracy: {0:f}'.format(accuracy_score))

Learn has many models, for example it is possible to create a linear classification model or even a Random Forest! It is also possible to create an highly customized model by providing a model function to an `Estimator` object. However if the model is highly complex maybe it is best to use plain TensorFlow.

### 3.2 Keras

> [Keras](https://keras.io/) is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. It was developed with a focus on enabling fast experimentation.



In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

Adding a layer to a Sequential model is very simple.

In [ ]:
model = Sequential()
model.add(Dense(hidden1_units, input_dim=IMAGE_PIXELS))
model.add(Activation('relu'))
model.add(Dense(10, activation='softmax'))

Next we need to "compile" the model. What it really does is to set up the computation and specify the learning algorithm.

In [ ]:
model.compile(optimizer='sgd', loss='mse')

Now we are ready for the training...

In [ ]:
model.fit(mnist.train.images, mnist.train.labels, epochs=50, batch_size=128, verbose=0)

...and prediction.

In [ ]:
test_label_pred = np.argmax(model.predict(mnist.test.images, batch_size=128), axis=1)
np.equal(test_label_pred,  np.argmax(mnist.test.labels, axis=1)).mean()

### 3.3 Considerations on MLP

The performance of the simple MLP is pretty good, but what if we want to improve it? Increasing the number of layers does not improve the accuracy, but why? This is known as the vanishing or exploding gradient problems. That is, the learning procedure based on gradients is unstable if we have many layers, resulting in the last layers to learn slowly. For this reason in order to train deeper network we must use another kind of network, the **Convolutional Neural Network**. In practice to avoid the vanishing gradient problem we use a specialized network with fewer parameters, that takes advantage of spatial locality of the images.

---

Visit [www.add-for.com](<http://www.add-for.com/IT>) for more tutorials and updates.

This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.